In [16]:
# Install necessary libraries
!pip install transformers
!pip install tokenizers
!pip install rdkit

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [17]:
import pandas as pd
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from rdkit import Chem

In [18]:
# Step 1: Load tokenizer and model directly from Hugging Face
tokenizer = PreTrainedTokenizerFast.from_pretrained("jonghyunlee/MolGPT_pretrained-by-ZINC15")
tokenizer.pad_token = "<pad>"
tokenizer.bos_token = "<bos>"
tokenizer.eos_token = "<eos>"

In [19]:
model = GPT2LMHeadModel.from_pretrained("jonghyunlee/MolGPT_pretrained-by-ZINC15")


In [20]:
def preprocess_data(file_path):
    df = pd.read_csv(file_path)
    
    # Remove the first row and use only the first column (SMILES)
    df = df.iloc[1:, 0]  # Skip the first row and select the first column
    
    return df

In [21]:
df = preprocess_data('inhibition_data.csv')


In [22]:
def tokenize_smiles(smiles_list):
    return tokenizer(smiles_list, padding=True, truncation=True, max_length=128, return_tensors="pt")

In [23]:
smiles_data = df.tolist()  # Convert the SMILES column to a list
tokenized_data = tokenize_smiles(smiles_data)

In [24]:
def generate_smiles(model, tokenizer, num_sequences=10, temperature=1.0):
    outputs = model.generate(
        max_length=128,
        num_return_sequences=num_sequences,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=temperature,
        return_dict_in_generate=True,
    )
    
    generated_smiles = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs.sequences]
    return generated_smiles

In [25]:
generated_smiles = generate_smiles(model, tokenizer, num_sequences=10, temperature=1.0)


In [26]:
def validate_smiles(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule:
        return True  # Valid SMILES
    else:
        return False  # Invalid SMILES

In [27]:

# Step 6: Print generated SMILES and validation result
for idx, smiles in enumerate(generated_smiles):
    is_valid = validate_smiles(smiles)
    print(f"Generated SMILES {idx + 1}: {smiles}")
    print(f"Valid: {is_valid}\n")

Generated SMILES 1:  O=C1OC[C@@H]2CN(C(=O)NCCc3cnccn3)CCN12
Valid: True

Generated SMILES 2:  CC(C)c1nnnn1CC(=O)N1CC[C@@H](NC(=O)c2cnns2)C1
Valid: True

Generated SMILES 3:  NC(=O)NC(=O)CCNC(=O)c1cc2ccncc2[nH]1
Valid: True

Generated SMILES 4:  CC(C)(C)OC(=O)c1cccc(NC(=O)C(=O)NC(CO)CO)c1
Valid: True

Generated SMILES 5:  C[C@H](NC(=O)Cn1cnc(-c2nn[nH]n2)n1)c1ccc2c(c1)OCCO2
Valid: True

Generated SMILES 6:  CCn1nnnc1N1CCN(Cc2nc(COC)n[nH]2)CC1
Valid: True

Generated SMILES 7:  O=C(C[C@@H]1NC(=O)NC1=O)NC1CN(C(=O)[C@@H]2[C@H]3CCC[C@H]32)C1
Valid: True

Generated SMILES 8:  CC[C@H](F)C(=O)NC[C@@H](O)CNC(=O)c1nccnc1N
Valid: True

Generated SMILES 9:  CC(=O)N1CCC[C@@H]1C(=O)N[C@H]1CCN([C@@H](C)C(=O)NC2CC2)C1
Valid: True

Generated SMILES 10:  COCC(=O)NCCC(=O)N(C)CCNC(=O)c1cnc(C)nc1C
Valid: True

